In [89]:
import numpy as np
import glob
import nltk
import re

In [83]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [36]:
def loadData(location):
    data = []
    for name in glob.glob(location):
        file = open(name, encoding="utf8", errors='ignore')
        s = file.read()
        data.append([s, location])
    return data

In [102]:
data = []
data += loadData('Dataset/business/*.txt')
data += loadData('Dataset/entertainment/*.txt')
data += loadData('Dataset/politics/*.txt')
data += loadData('Dataset/sport/*.txt')
data += loadData('Dataset/tech/*.txt')

x_train = []
y_train = []

for i in range(len(data)):
    data[i][0] = re.sub('\n', '. ', data[i][0])
    data[i][0] = data[i][0].lower()
    data[i][0] = re.sub(r'[^a-z]', ' ', data[i][0])
    x_train.append(data[i][0])
    
    data[i][1] = re.sub('Dataset', '', data[i][1])
    data[i][1] = re.sub('.txt', '', data[i][1])
    data[i][1] = re.sub('/', '', data[i][1])
    data[i][1] = re.sub(r'[^a-z]', '', data[i][1])
    y_train.append(data[i][1])

In [103]:
num_words = 10000
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)

x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_train_tokens = pad_sequences(x_train_tokens, maxlen=5000, padding='pre', truncating='pre')

def one_hot_encoder(y):
    list_label = list(set(y))
    label = np.zeros([len(y), len(list_label)])
    for i in range(len(y)):
        label[i][list_label.index(y[i])] = 1
    return label

y_train = one_hot_encoder(y_train)

In [104]:
x_train_tokens

array([[   0,    0,    0, ...,    3,   10, 1233],
       [   0,    0,    0, ...,    5, 1998, 2308],
       [   0,    0,    0, ...,    6,    1,  441],
       ...,
       [   0,    0,    0, ...,  449,  141,  227],
       [   0,    0,    0, ...,   73,    6, 2696],
       [   0,    0,    0, ...,   14,    1,  368]], dtype=int32)

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
model = Sequential()
max_tokens = 5000
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(5, activation='softmax'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
y_train = np.array(y_train)
model.fit(x_train_tokens, y_train)

/Users/fahmisalman/.conda/envs/Anaconda3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 416/2225 [====>.........................] - ETA: 13:42 - loss: 0.5018 - acc: 0.8000